# Data Crawl and Clean
---
Yufan Zhang (yz605@duke.edu)

Junyi Tao (jt328@duke.edu)

Ran Peng (rp231@duke.edu)

In [13]:
import requests
import json
import os
import pandas as pd
import numpy as np
import re

## 1. Use DKU Faculty Members' emails to batch search for DUIDs

Duke  IdM Tools-[LDAP Batch Search by Identifer](https://idm.oit.duke.edu/tools/ldap)

In [5]:
dku_emails = pd.read_csv('../data/dku_emails.csv')

s_temp = ''
for i in range(len(dku_emails)):
    s_temp += dku_emails.iloc[i]['Email']+'\n'
    
with open('../data/dku_duid_list.txt','w') as f:
    f.write(s_temp)

In [7]:
LDAP_data = pd.read_csv('../data/2022-4-11-ldap.csv')
LDAP_data.head()

,dn,Duke UniqueID (duDukeID),NetID (uid),First Name (givenName),Last Name (sn),Common Name (cn),Display Name (dn),Nickname (eduPersonNickname),Title (title),Affiliation(s) (eduPersonAffiliation),Primary Affiliation (eduPersonPrimaryAffiliation),Email Address (mail),Telephone Number (telephoneNumber),Company Code Description (duSAPCompanyDesc),Organizational Unit (ou),SAP Org Unit (duSAPOrgUnit),Email Reciept and Telephone Number Privacy Flag (duUserPrivacyRequest),Postal Address (PostalAddress),Post Office Box (PostOfficeBox),Generational Name Qualifier (generationQualifier)
0,duLDAPKey=dc808dee-1dd1-11b2-b6a9-9442cd793b76...,480813,djs26,Daniel,Stephens,Daniel Stephens Daniel J Stephens,"Daniel Stephens, Ph.D.",Daniel,Assistant Professor of Philosophy at Duke Kuns...,alumni affiliate faculty,faculty,daniel.stephens@duke.edu,+1 919 668 0377,Duke University,Dku Appointments,51032158,NaN,NaN,NaN,NaN
1,duLDAPKey=3a750dd4-1dd2-11b2-8a54-9442cd793b76...,501216,jc254,Jung,Choi,Jung Choi Jung E Choi,"Prof Jung Choi, Ph.D.",Jung,Assistant Professor of Arts History and Visual...,alumni affiliate,affiliate,jung.choi@dukekunshan.edu.cn,+1 919 684 2224,Duke University,NaN,51032158,NaN,NaN,NaN,NaN
2,duLDAPKey=0f2ac190-1dd2-11b2-b7bc-e02c52381c12...,663997,ba83,Benjamin,Anderson,Benjamin Anderson,Benjamin Anderson,Benjamin,Assistant Professor of Global Health at Duke K...,affiliate,affiliate,benjamin.anderson2@dukekunshan.edu.cn,NaN,Duke University,NaN,51032158,NaN,NaN,NaN,NaN
3,duLDAPKey=7ff6272e-1dd2-11b2-b7c3-e02c52381c12...,664012,dbs25,Donald,Snow,Donald Snow Don Snow,Don Snow,Don,Director of Language and Writing Program at Du...,affiliate,affiliate,don.snow@dukekunshan.edu.cn,NaN,Duke University,NaN,51032158,NaN,NaN,NaN,NaN
4,duLDAPKey=e456d7dc-f85d-11e5-a18d-c3e21a144c1c...,715779,kmc104,Kaley,Clements,Kaley Clements,Kaley Clements,Kaley,Assistant Professor of Media and Arts at Duke ...,alumni affiliate faculty,faculty,kaley.clements@duke.edu,+1 919 684 1958,Duke University,Dku Appointments,51032158,NaN,NaN,NaN,NaN


In [18]:
duids_dku = LDAP_data[['Duke UniqueID (duDukeID)', 'Common Name (cn)']]
duids_dku.columns = ['id', 'name']
duids_dku = duids_dku[['name', 'id']]
duids_dku.to_csv('../data/duids_dku.csv')
duids_dku.head()

,name,id
0,Daniel Stephens Daniel J Stephens,480813
1,Jung Choi Jung E Choi,501216
2,Benjamin Anderson,663997
3,Donald Snow Don Snow,664012
4,Kaley Clements,715779


## 2. Use the DUIDs to crawl JSON via Scholars widgets API

[Scholars@Duke Swagger widgets API](https://scholars.duke.edu/widgets/docs/#!/widgets/get_api_v0_9_people_complete_count_format)

### Load DUIDs

In [19]:
duids_all = pd.read_csv('../data/duids_all.csv')
duids_all = duids_all[pd.notnull(duids_all['id'])].id.apply(lambda x: str(int(x)).zfill(7))
all = tuple(duids_all)
print(len(all))

# duids_dku = pd.read_csv('../data/duids_dku.csv')
duids_dku = duids_dku[pd.notnull(duids_dku['id'])].id.apply(lambda x: str(int(x)).zfill(7))
dku = tuple(duids_dku)
print(len(dku))

duids = list(set(all + dku))
print(len(duids))

6927
104
6910


In [20]:
duids

['0278212',
 '0099834',
 '0488434',
 '0669318',
 '0434292',
 '0147385',
 '0717248',
 '0641212',
 '0212431',
 '0081429',
 '0280987',
 '0327317',
 '0269217',
 '0114935',
 '0096574',
 '0969226',
 '0508955',
 '0314731',
 '1052902',
 '0697523',
 '0414076',
 '0280297',
 '0819615',
 '0696721',
 '0202984',
 '0304146',
 '0073333',
 '0296928',
 '0982477',
 '0284583',
 '0806816',
 '1051511',
 '0610737',
 '0097727',
 '0310314',
 '0444890',
 '1114625',
 '0842646',
 '0414195',
 '0035869',
 '0112831',
 '0217002',
 '0987980',
 '0330994',
 '0075578',
 '0113941',
 '0193175',
 '0020675',
 '0320920',
 '0511609',
 '0705587',
 '1114752',
 '1057410',
 '0535456',
 '0520056',
 '0666575',
 '1190303',
 '0115165',
 '0800754',
 '0283960',
 '0081108',
 '0071342',
 '0332707',
 '0060651',
 '0482452',
 '1059096',
 '0706300',
 '0099133',
 '1072677',
 '0563576',
 '0114339',
 '0642805',
 '0593762',
 '0631346',
 '0304067',
 '0118580',
 '0226708',
 '1049341',
 '0483045',
 '0052951',
 '1191190',
 '0333631',
 '0473836',
 '05

### Crawl JSON data

Data Mode
- Complete
- Publication

In [41]:
def crawl(duid, data_mode):
    url = 'https://scholars.duke.edu/widgets/api/v0.9/people/{}/5.json?uri={}'.format(data_mode, duid)
    headers = {"Accepct": "application/json"}
    response = requests.get(url=url, headers=headers)
    if response.status_code == 404:
        return ('No Response', duid)
    json_info = response.json()
    return json_info

def save_json(data_mode, duid, content):
    dir_str = '../data/scholars_duke_new/{}/{}.json'.format(data_mode, duid)
    with open(dir_str, 'w') as outfile:
        json.dump(content, outfile)

In [21]:
def get_all_json(duids, data_mode):
    '''
        duids: list of duids
        data_mode: 'complete', 'publications', 'courses', 'educations'
    '''
    counter = 0
    for i in range(len(duids)):
        duid = duids[i]
        # name = duid_df.loc[i, 'name']
        no_info_list = list()
        
        json_content = crawl(duid, data_mode)
            
        if type(json_content) == tuple:
            no_info_list.append(json_content[1])
        else:
            counter += 1
            save_json(data_mode, duid, json_content)
        
        if i % 500 == 0:
            print('============== {}/{} completed =============='.format(i, len(duids)))
    print('============== {} JSON files containing {} infomation in total crawled =============='.format(counter, data_mode))

In [45]:
get_all_json(duids, 'complete')

============== 0/6910 completed ==============
============== 500/6910 completed ==============
============== 1000/6910 completed ==============
============== 1500/6910 completed ==============
============== 2000/6910 completed ==============
============== 2500/6910 completed ==============
============== 3000/6910 completed ==============
============== 3500/6910 completed ==============
============== 4000/6910 completed ==============
============== 4500/6910 completed ==============
============== 5000/6910 completed ==============
============== 5500/6910 completed ==============
============== 6000/6910 completed ==============
============== 6500/6910 completed ==============
============== 6801 JSON files containing complete infomation in total crawled ==============


## 3. Clean the JSON files

In [22]:
def get_education(d):
    '''
        Input:
            d: a dict containing the education information, ori_d['educations'][i]
        Output:
            a dict containing the degree,institution, end_year
    '''
    out = dict()
    if 'degree' in d['attributes'].keys():
        out['degree'] = d['attributes']['degree']
    else:
        out['degree'] = None
    
    if 'institution' in d['attributes'].keys():
        out['institution'] = d['attributes']['institution']
    else:
        out['institution'] = None
        
    if 'endDate' in d['attributes'].keys():
        out['end_year'] = d['attributes']['endDate']
    else:
        out['end_year'] = None
    return out


def get_publication(d):
    '''
        Input:
            d: a dict containing the publication information, ori_d['publications'][i]
        Output:
            a dict containing the title, authors, year
    '''
    out = dict()
    out['title'] = d['label']
    if 'authorList' not in d['attributes'].keys():
        out['authors'] = None
    else:
        out['authors'] = d['attributes']['authorList']
        
    if 'year' not in d['attributes'].keys():
        out['year'] = None
    else:
        out['year'] = d['attributes']['year']
    return out


def get_courses(d):
    '''
        Input:
            d: a dict containing the courses information, ori_d['courses'][i]
        Output:
            a string containing the course_label
    '''
    out = d['label']
    return out

In [23]:
def get_info(json_file_name):
    '''
        Input:
            json_file_name: the name of the json file
        Output:
            a dict containing faculty information, counts for the educations, publications and courses
    '''
    f = open('../data/scholars_duke_new/complete/' + json_file_name, 'r')
    raw_d = dict(json.load(f))
    ind_d = dict()
    duid = json_file_name.split('.')[0]
    
    ind_d['duid'] = duid                                                    # duid
    ind_d['name'] = raw_d['label']                                          # name
    ind_d['uri'] = raw_d['uri']                                             # uri
    ind_d['title'] = raw_d['title']                                         # title
    if len(raw_d['positions']) > 0:
        ind_d['school'] = raw_d['positions'][0]['attributes']['schoolLabel']        # school
    else:
        ind_d['school'] = None
    from_dku = ind_d['school'] == 'Duke Kunshan University'
    
    educations = list()
    for edu_d in raw_d['educations']:
        educations.append(get_education(edu_d))
    ind_d['educations'] = educations                                       # educations
        
    publications = list()
    for pub_d in raw_d['publications']:
        publications.append(get_publication(pub_d))
    ind_d['publications'] = publications                                   # publications
    
    courses = list()
    for cou_d in raw_d['courses']:
        courses.append(get_courses(cou_d))
    ind_d['course'] = courses                                              # courses
    
    return ind_d, len(educations), len(publications), len(courses), from_dku

In [24]:
def clean_json():
    '''
    Output:
        a dictionary containing all the information
        {duid : {   duid : string, 
                    name : string,
                    uri : string, 
                    title : string,
                    school : string,
                    educations : [
                        {
                            degree: string,
                            institution : string,
                            end_year : string
                        }
                    ], 
                    publications : [
                        {
                            title : string,
                            authors : string,
                            year : string
                        }
                    ]
                    courses : [
                        string
                    ]
                }
            }
    '''
    out_d = dict()
    out_d_dku = dict()
    json_file_names = os.listdir('../data/scholars_duke_new/complete/')
    counter = 0
    edu_counter, pub_counter, cou_counter = 0, 0, 0
    edu_sum, pub_sum, cou_sum = 0, 0, 0
    
    for json_file_name in json_file_names:
        if re.match(r"[0-9]+(.json)", json_file_name):
            # print('=== Processing {} ==='.format(json_file_name))
            
            duid = json_file_name.split('.')[0]
            
            # Get individual information
            ind_d, n_edu, n_pub, n_cou, from_dku = get_info(json_file_name)
            
            ## save to out_d_dku if the person is a DKU faculty
            if from_dku:
                out_d_dku[duid] = ind_d
                
            out_d[duid] = ind_d
            
            # Log counter
            if n_edu > 0:
                edu_counter += 1
                edu_sum += n_edu
            if n_pub > 0:
                pub_counter += 1
                pub_sum += n_pub
            if n_cou > 0:
                cou_counter += 1
                cou_sum += n_cou
            counter += 1
            
            if counter % 500 == 0:
                print('============= {}/{} ============='.format(counter, len(json_file_names)))
    
    print('=== Summary ===')   
    print('count of scholars who have educations info: {}/{}'.format(edu_counter, len(json_file_names)))
    print('total education info pieces: {}\n'.format(edu_sum))
    print('count of scholars who have publications info: {}/{}'.format(pub_counter, len(json_file_names)))
    print('total publication info pieces: {}\n'.format(pub_sum))
    print('count of scholars who have courses info: {}/{}'.format(cou_counter, len(json_file_names)))
    print('total course info pieces: {}\n'.format(cou_sum))
    
    print('DKU/Duke: {}/{}'.format(len(out_d_dku), len(out_d)))
    
    with open("../cleaned_scholars.json", "w") as outfile:
        json.dump(out_d, outfile)
    print('=== All Info: Done! ===')
    
    with open("../cleaned_scholars_dku.json", "w") as outfile:
        json.dump(out_d_dku, outfile)
    print('=== DKU Info: Done! ===')
    
    return out_d, out_d_dku, [edu_counter, pub_counter, cou_counter], [edu_sum, pub_sum, cou_sum]

In [70]:
out_d, out_d_dku, counters, sums = clean_json()

============= 500/6801 =============
============= 1000/6801 =============
============= 1500/6801 =============
============= 2000/6801 =============
============= 2500/6801 =============
============= 3000/6801 =============
============= 3500/6801 =============
============= 4000/6801 =============
============= 4500/6801 =============
============= 5000/6801 =============
============= 5500/6801 =============
============= 6000/6801 =============
============= 6500/6801 =============
=== Summary ===
count of scholars who have educations info: 6665/6801
total education info pieces: 14403

count of scholars who have publications info: 4425/6801
total publication info pieces: 374192

count of scholars who have courses info: 2493/6801
total course info pieces: 15268

DKU/Duke: 171/6801
=== All Info: Done! ===
=== DKU Info: Done! ===
